In [ ]:
%pylab inline
import pandas as pd
import os

# Just use 1 GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
from pywsi.io import WSIReader
from pywsi.morphology import TissuePatch
from matplotlib.patches import Polygon

from shapely.geometry import Point as shapelyPoint
from shapely.geometry import box as shapelyRectangle

from pywsi.io.operations import get_annotation_bounding_boxes, get_annotation_polygons, translate_and_scale_object
from pywsi.io.operations import translate_and_scale_polygon
from openslide.deepzoom import DeepZoomGenerator
from multiprocessing import Pool

import os
import glob
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray, gray2rgb
from shapely.geometry import Polygon as shapelyPolygon
import openslide

from tqdm import tqdm_notebook, tqdm
import cv2

from pywsi.io.operations import get_annotation_bounding_boxes, get_annotation_polygons, \
    poly2mask, translate_and_scale_polygon, read_as_rgb

from pywsi.morphology.patch_extractor import TissuePatch
from pywsi.morphology.mask import mpl_polygon_to_shapely_scaled, get_common_interior_polygons

from keras.models import Sequential
from keras.layers import Lambda, Dropout
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from keras.callbacks import ModelCheckpoint

from pywsi.io.tiling import generate_tiles, get_all_patches_from_slide
import matplotlib.gridspec as gridspec
from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook
from matplotlib import cm


NUM_CLASSES = 2 # not_tumor, tumor
BATCH_SIZE = 32

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(256, 256, 3)))
model.add(Convolution2D(100, (5, 5), strides=(2, 2), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Convolution2D(200, (5, 5), strides=(2, 2), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Convolution2D(300, (3, 3), activation='elu', padding='same'))
model.add(Convolution2D(400, (3, 3), activation='elu',  padding='same'))
model.add(Dropout(0.1))
model.add(Convolution2D(400, (3, 3), activation='elu',  padding='same'))
model.add(Convolution2D(300, (3, 3), activation='elu',  padding='same'))
model.add(Dropout(0.1))
model.add(Convolution2D(2, (1, 1))) # this is called upscore layer for some reason?
model.add(Conv2DTranspose(2, (31, 31), strides=(16, 16), activation='softmax', padding='same'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.load_weights('weights-improvement-11-0.97.hdf')

In [ ]:
# Let's try on tumor_076 samples
def predict_from_model(patch, model):
    """Predict which pixels are tumor.
    
    input: patch: 256x256x3, rgb image
    input: model: keras model
    output: prediction: 256x256x1, per-pixel tumor probability
    """
    
    prediction = model.predict(patch.reshape(1, 256, 256, 3))
    prediction = prediction[:, :, :, 1].reshape(256, 256)
    return prediction



In [ ]:
def plot_blend(patch, prediction, ax, alpha=0.75):
    """alpha blend patch and prediction.
    https://matplotlib.org/examples/pylab_examples/layer_images.html
    
    input: patch: 256x256x3, rgb image
    input: prediction: 256x256x1, per-pixel tumor probability
    input: ax: maplotlib Axes object
    input: alpha: alpha blend
    """
    
    dx, dy = 0.05, 0.05
    x = np.arange(0, patch.shape[1] - 1, dx)
    y = np.arange(0, patch.shape[0] - 1, dy)
    xmin, xmax, ymin, ymax = np.amin(x), np.amax(x), np.amin(y), np.amax(y)
    extent = xmin, xmax, ymin, ymax

    # fig = plt.figure(frameon=False, figsize=(10, 5))
    Z1 = rgb2gray(patch)
    Z2 = prediction

    im1 = ax.imshow(Z1, cmap='gray', extent=extent)
    im2 = ax.imshow(Z2, cmap='coolwarm', alpha=alpha, vmin=0.0, vmax=1.0,
                     extent=extent)
    ax.axis('off');

def plot_patch_with_pred(patch, truth, prediction, title_str='', alpha=0.6):
    """
    input: patch: 256x256x3, rgb image
    input: truth: 256x256x2, onehot output classes (not_tumor, tumor)
    input: prediction: 256x256x1, per-pixel tumor probability
    """
    gs = gridspec.GridSpec(2, 4, width_ratios=[10, 10, 19, 1])
    ax0 = plt.subplot(gs[0, 0])
    ax1 = plt.subplot(gs[0, 1])
    ax2 = plt.subplot(gs[1, 0])
    ax3 = plt.subplot(gs[1, 1])
    ax4 = plt.subplot(gs[:, 2])
    axc = plt.subplot(gs[:, 3])

    ax0.imshow(patch);
    ax0.set_title('Original')
    
    ax1.imshow(truth.argmax(axis=2), cmap='gray', vmin=0, vmax=1);
    ax1.set_title('Truth mask (white=tumor, black=not_tumor)')
    
    p = ax2.imshow(prediction, cmap='coolwarm', vmin=0, vmax=1);
    ax2.set_title('Prediction heatmap')

    ax3.imshow((prediction > 0.5).astype(np.int), cmap='gray', vmin=0, vmax=1);
    ax3.set_title('Prediction mask (white=tumor, black=not_tumor)')
    
    plot_blend(patch, prediction, ax4, alpha)
    ax4.set_title('Original+Prediction blend')
    
    fig = plt.gcf()
    fig.set_size_inches(20, 10)
    fig.suptitle(title_str)
    fig.colorbar(p, cax=axc, orientation="vertical")
    axc.set_title('Probability pixel is tumor')

In [ ]:
def predict_batch_from_model(patches, model):
    """Predict which pixels are tumor.
    
    input: patch: `batch_size`x256x256x3, rgb image
    input: model: keras model
    output: prediction: 256x256x1, per-pixel tumor probability
    """
    predictions = model.predict(patches)
    predictions = predictions[:, :, :, 1]
    return predictions
"""
validation_samples = len(tumor_076.index)
validation_generator = generate_tiles(tumor_076, BATCH_SIZE)
validation_steps = np.ceil((validation_samples) / BATCH_SIZE)

confusion_mtx = np.zeros((2, 2))

for i in tqdm_notebook(range(int(validation_steps))):
    X, y  = next(validation_generator)
    preds = predict_batch_from_model(X, model)
    
    y_true = y[:, :, :, 1].ravel()
    y_pred = np.uint8(preds > 0.5).ravel()
    
    confusion_mtx += confusion_matrix(y_true, y_pred, labels=[0, 1])
"""

In [ ]:
tumor_df = pd.read_table('/Z/personal-folders/interns/saket/histopath_data/patches_dataframe/training/tumor/master_df.tsv')


In [ ]:
tumor_076 = tumor_df[tumor_df.uid=='tumor_076']
tumor_082 = tumor_df[tumor_df.uid=='tumor_082']
tumor_002 = tumor_df[tumor_df.uid=='tumor_002']

In [ ]:
tumor_002.tile_loc = [eval(x) for x in tumor_002.tile_loc]

In [ ]:
sample_gen = generate_tiles(tumor_002.sample(32, random_state=42), 32, shuffle=False)

example_X, example_y  = next(sample_gen)

In [ ]:
example_patch = example_X[1]
example_truth = example_y[1]

prediction = predict_from_model(example_patch, model)
plot_patch_with_pred(example_patch, example_truth, prediction, title_str='Example Tumor Patch')


In [ ]:
output_dir = '/Z/personal-folders/interns/saket/histopath_data/prediction_heatmaps/tumor_002'
os.makedirs(output_dir, exist_ok=True)

alpha = 0.5
slide_path = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/tumor/tumor_002.tif'
json_filepath = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/lesion_annotations_json/tumor_002.json'
all_samples = get_all_patches_from_slide(slide_path, json_filepath, False, 256)
slide = WSIReader(slide_path, 40)

n_samples = len(all_samples)
n_cols = int(slide.dimensions[0] / 256)
n_rows = int(slide.dimensions[1] / 256)
#assert n_cols * n_rows == n_samples

thumbnail = slide.get_thumbnail((n_cols, n_rows))
thumbnail = np.array(thumbnail)

# batch_size = n_cols
batch_size = 32
output_thumbnail_preds = list()
    
for offset in tqdm_notebook(list(range(0, n_samples, batch_size))):
    batch_samples = all_samples.iloc[offset:offset+batch_size]
    #batch_samples.loc[: 'tile_loc'] = [eval(x) for x in batch_samples.tile_loc]
    png_fnames = batch_samples.tile_loc.apply(lambda coord: os.path.join(output_dir,
                                                                         '{}_{}.png'.format(coord[1], coord[0])))
    
    X, _ = next(generate_tiles(batch_samples, batch_size, shuffle=False))
    
    if batch_samples.is_tissue.nunique() == 1 and batch_samples.iloc[0].is_tissue == False:
        # all patches in this row do not have tissue, skip them all
        output_thumbnail_preds.append(np.zeros(batch_size, dtype=np.float32))
        
        # output pngs
        for i, png_fname in enumerate(png_fnames):
            plt.imsave(png_fname, X[i])
    else:
        # make predictions
        preds = predict_batch_from_model(X, model)
        output_thumbnail_preds.append(preds.mean(axis=(1,2)))
        
        # overlay preds
        # save blended imgs
        for i, png_fname in enumerate(png_fnames):
            pred_i = preds[i]
            X_i = X[i]
            #output_img = rgb2gray(X_i)
            #output_img2 = gray2rgb(output_img.copy())

            #overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            output_img = cv2.cvtColor(X_i, cv2.COLOR_RGB2GRAY)
            output_img2 = cv2.cvtColor(output_img.copy(), cv2.COLOR_GRAY2RGB)

            overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            blended = cv2.addWeighted(overlay, alpha, output_img2, 1-alpha, 0, output_img)
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            #blended = np.clip(blended, 0, 255)
            plt.imsave(png_fname, blended)
        

output_thumbnail_preds = np.array(output_thumbnail_preds)

In [ ]:
# Remove first 10 and last 10 entries
output_thumbnail_preds_reshaped = output_thumbnail_preds[1:-1,:].reshape(836, 392)#reshape(n_rows, n_cols)
f, axes = plt.subplots(1, 2, figsize=(40, 18))
ax = axes.flatten()
plot_blend(thumbnail, output_thumbnail_preds, ax=ax[0])

In [ ]:
output_dir = '/Z/personal-folders/interns/saket/histopath_data/prediction_heatmaps/tumor_009'
os.makedirs(output_dir, exist_ok=True)

alpha = 0.5
slide_path = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/tumor/tumor_009.tif'
json_filepath = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/lesion_annotations_json/tumor_009.json'
all_samples = get_all_patches_from_slide(slide_path, json_filepath, False, 256)
slide = WSIReader(slide_path, 40)

n_samples = len(all_samples)
n_cols = int(slide.dimensions[0] / 256)
n_rows = int(slide.dimensions[1] / 256)
assert n_cols * n_rows == n_samples

thumbnail = slide.get_thumbnail((n_cols, n_rows))
thumbnail = np.array(thumbnail)

# batch_size = n_cols
batch_size = 32
output_thumbnail_preds = list()
    
for offset in tqdm(list(range(0, n_samples, batch_size))):
    batch_samples = all_samples.iloc[offset:offset+batch_size]
    #batch_samples.loc[: 'tile_loc'] = [eval(x) for x in batch_samples.tile_loc]
    png_fnames = batch_samples.tile_loc.apply(lambda coord: os.path.join(output_dir,
                                                                         '{}_{}.png'.format(coord[1], coord[0])))
    
    X, _ = next(generate_tiles(batch_samples, batch_size, shuffle=False))
    
    if batch_samples.is_tissue.nunique() == 1 and batch_samples.iloc[0].is_tissue == False:
        # all patches in this row do not have tissue, skip them all
        output_thumbnail_preds.append(np.zeros(batch_size, dtype=np.float32))
        
        # output pngs
        for i, png_fname in enumerate(png_fnames):
            plt.imsave(png_fname, X[i])
    else:
        # make predictions
        preds = predict_batch_from_model(X, model)
        output_thumbnail_preds.append(preds.mean(axis=(1,2)))
        
        # overlay preds
        # save blended imgs
        for i, png_fname in enumerate(png_fnames):
            pred_i = preds[i]
            X_i = X[i]
            #output_img = rgb2gray(X_i)
            #output_img2 = gray2rgb(output_img.copy())

            #overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            output_img = cv2.cvtColor(X_i, cv2.COLOR_RGB2GRAY)
            output_img2 = cv2.cvtColor(output_img.copy(), cv2.COLOR_GRAY2RGB)

            overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            blended = cv2.addWeighted(overlay, alpha, output_img2, 1-alpha, 0, output_img)
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            #blended = np.clip(blended, 0, 255)
            plt.imsave(png_fname, blended)
        

output_thumbnail_preds = np.array(output_thumbnail_preds)

In [ ]:
#def process_batch(batch_samples):
output_dir = '/Z/personal-folders/interns/saket/histopath_data/prediction_heatmaps/tumor_009'
os.makedirs(output_dir, exist_ok=True)

alpha = 0.5
slide_path = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/tumor/tumor_009.tif'
json_filepath = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/lesion_annotations_json/tumor_009.json'
all_samples = get_all_patches_from_slide(slide_path, json_filepath, False, 256)
slide = WSIReader(slide_path, 40)

n_samples = len(all_samples)
n_cols = int(slide.dimensions[0] / 256)
n_rows = int(slide.dimensions[1] / 256)
assert n_cols * n_rows == n_samples

thumbnail = slide.get_thumbnail((n_cols, n_rows))
thumbnail = np.array(thumbnail)

# batch_size = n_cols
batch_size = 32
output_thumbnail_preds = list()
    
    
def process_batch(args):
    idx, batch_samples = args
    png_fnames = batch_samples.tile_loc.apply(lambda coord: os.path.join(output_dir,
                                                                         '{}_{}.png'.format(coord[1], coord[0])))
    output_thumbnail_pred = None
    X, _ = next(generate_tiles(batch_samples, batch_size, shuffle=False))
    if batch_samples.is_tissue.nunique() == 1 and batch_samples.iloc[0].is_tissue == False:
        # all patches in this row do not have tissue, skip them all
        output_thumbnail_pred = np.zeros(batch_size, dtype=np.float32)
        
        # output pngs
        for i, png_fname in enumerate(png_fnames):
            plt.imsave(png_fname, X[i])
    else:
        # make predictions
        preds = predict_batch_from_model(X, model)
        output_thumbnail_pred = preds.mean(axis=(1,2))
        
        # overlay preds
        # save blended imgs
        
        for i, png_fname in enumerate(png_fnames):
            pred_i = preds[i]
            X_i = X[i]
            #output_img = rgb2gray(X_i)
            #output_img2 = gray2rgb(output_img.copy())

            #overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            output_img = cv2.cvtColor(X_i, cv2.COLOR_RGB2GRAY)
            output_img2 = cv2.cvtColor(output_img.copy(), cv2.COLOR_GRAY2RGB)

            overlay = np.uint8(cm.viridis(pred_i) * 255)[:,:,:3]
            blended = cv2.addWeighted(overlay, alpha, output_img2, 1-alpha, 0, output_img)
            #blended = overlay*alpha + output_img2 *(1-alpha) +  0
            #blended = np.clip(blended, 0, 255)
            plt.imsave(png_fname, blended)
    return idx, output_thumbnail_pred

all_batch_samples = []
for offset in tqdm_notebook(list(range(0, n_samples, batch_size))):
    all_batch_samples.append(all_samples.iloc[offset:offset+batch_size])

In [ ]:
total = len(list(range(0, n_samples, batch_size)))

output_thumbnail_preds = []
output_thumbnail_idx = []
with tqdm_notebook(total=total) as pbar:
    with Pool(processes=8) as p:
        results = p.imap_unordered(process_batch, enumerate(all_batch_samples))
        for idx, result in results:
            output_thumbnail_preds.append(result)
            output_thumbnail_idx.append(idx)
            pbar.update()
    #output_thumbnail_pred = list(tqdm_notebook(p.imap(process_batch, all_batch_samples), total=total))
        #for i, output_thumbnail_pred in enumerate(p.imap(process_batch, all_batch_samples)):
        #    output_thumbnail_preds.append(output_thumbnail_pred)
        #    pbar.update()

output_thumbnail_preds = np.array(output_thumbnail_preds)

In [ ]:
from functools import reduce

def factors(n):    
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(pow(n, 0.5) + 1)) if n % i == 0)))

In [ ]:
10241 * 32/392